# 快来选一顿好吃的年夜饭：看看如何自定义数据集，实现文本分类中的情感分析任务

情感分析是自然语言处理领域一个老生常谈的任务。句子情感分析目的是为了判别说者的情感倾向，比如在某些话题上给出的的态度明确的观点，或者反映的情绪状态等。情感分析有着广泛应用，比如电商评论分析、舆情分析等。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/febb8a1478e34258953e56611ddc76cd20b412fec89845b0a4a2e6b9f8aae774" hspace='10'/> <br />
</p>

# 题目：
将lstm网络替换成其他网络。可参考[seq2vec介绍](https://aistudio.baidu.com/aistudio/projectdetail/1283423)

提示位置：self.lstm_encoder = ppnlp.seq2vec.LSTMEncoder()

# 环境介绍

- PaddlePaddle框架，AI Studio平台已经默认安装最新版2.0。

- PaddleNLP，深度兼容框架2.0，是飞桨框架2.0在NLP领域的最佳实践。

这里使用的是beta版本，马上也会发布rc版哦。AI Studio平台后续会默认安装PaddleNLP，在此之前可使用如下命令安装。

In [1]:
# 下载paddlenlp
!pip install --upgrade paddlenlp==2.0.0b4 -i https://pypi.org/simple

     |████████████████████████████████| 174kB 125kB/s eta 0:00:01
ERROR: paddlehub 2.0.4 has requirement paddlenlp>=2.0.0rc5, but you'll have paddlenlp 2.0.0b4 which is incompatible.
  Found existing installation: paddlenlp 2.0.0rc7
    Uninstalling paddlenlp-2.0.0rc7:
      Successfully uninstalled paddlenlp-2.0.0rc7


查看安装的版本

In [2]:
import paddle
import paddlenlp

print(paddle.__version__, paddlenlp.__version__)


2.0.1 2.0.0b4


## PaddleNLP和Paddle框架是什么关系？

![]()

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/165924e86d9f4b5fa5d6fdee9e8496bf01be524e61f341b3879aceba48ae80fb" width = "300" height = "250"  hspace='10'/> <br />
</p><br></br>

- Paddle框架是基础底座，提供深度学习任务全流程API。PaddleNLP基于Paddle框架开发，适用于NLP任务。

PaddleNLP中数据处理、数据集、组网单元等API未来会沉淀到框架`paddle.text`中。


- 代码中继承
`class TSVDataset(paddle.io.Dataset)`

## 使用飞桨完成深度学习任务的通用流程

- 数据集和数据处理  
paddle.io.Dataset   
paddle.io.DataLoader   
paddlenlp.data   

- 组网和网络配置

paddle.nn.Embedding   
paddlenlp.seq2vec
paddle.nn.Linear   
paddle.tanh

paddle.nn.CrossEntropyLoss    
paddle.metric.Accuracy   
paddle.optimizer   

model.prepare    

- 网络训练和评估   
model.fit   
model.evaluate   

- 预测
model.predict   

注意：建议在GPU下运行。

In [3]:
import numpy as np
from functools import partial

import paddle.nn as nn
import paddle.nn.functional as F
import paddlenlp as ppnlp
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.datasets import MapDatasetWrapper

from utils import load_vocab, convert_example

# 数据集和数据处理

## 自定义数据集

映射式(map-style)数据集需要继承`paddle.io.Dataset`

- `__getitem__`: 根据给定索引获取数据集中指定样本，在 paddle.io.DataLoader 中需要使用此函数通过下标获取样本。

- `__len__`: 返回数据集样本个数， paddle.io.BatchSampler 中需要样本个数生成下标序列。



In [4]:
class SelfDefinedDataset(paddle.io.Dataset):
    def __init__(self, data):
        super(SelfDefinedDataset, self).__init__()
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)
        
    def get_labels(self):
        return ["0", "1","2"]

def txt_to_list(file_name):
    res_list = []
    for line in open(file_name):
        res_list.append(line.strip().split('\t'))
    return res_list

trainlst = txt_to_list('train.txt')
devlst = txt_to_list('dev.txt')
testlst = txt_to_list('test.txt')

# 通过get_datasets()函数，将list数据转换为dataset。
# get_datasets()可接收[list]参数，或[str]参数，根据自定义数据集的写法自由选择。
# train_ds, dev_ds, test_ds = ppnlp.datasets.ChnSentiCorp.get_datasets(['train', 'dev', 'test'])
train_ds, dev_ds, test_ds = SelfDefinedDataset.get_datasets([trainlst, devlst, testlst])



看看数据长什么样

In [5]:
label_list = train_ds.get_labels()
print(label_list)

for i in range(10):
    print (train_ds[i])

['0', '1', '2']
['环境不错，叉烧包小孩爱吃，三色煎糕很一般', '2']
['刚来的时候让我们等位置，我们就在门口等了十分钟左右，没有见到有人离开，然后有工作人员让我们上来二楼，上来后看到有十来桌是没有人的，既然有位置，为什么非得让我们在门口等位！！！本以为有座位后就可以马上吃饭了，让人内心崩溃的是点菜就等了十分钟才有人有空过来理我们。上菜更是郁闷，端来了一锅猪肚鸡，但是没人开火，要点调味料也没人管，服务质量太差，之前来过一次觉得还可以，这次让我再也不想来这家店了！真心失望', '0']
['口味还可以服务真的差到爆啊我来过45次真的次次都只给差评东西确实不错但你看看你们的服务还收服务费我的天干蒸什么的60块比太古汇翠园还贵主要是没人收台没人倒水谁还要自己倒我的天给你服务费还什么都自己干我接受不了钱花了服务没有实在不行', '0']
['出品不错老字号就是好有山有水有树有鱼赞', '1']
['在江南大道这间～服务态度好差，食物出品平凡，应该唔会再去了', '1']
['我是个挺食得咸的人，但那个黑糖叉烧真是太咸了。招牌的菠萝包，分量好大，可是味道呢，只不过是普通酥皮包里面加了陷，而且那个陷，食不到菠萝，食不到菠萝味，只是一堆糖酱。水鬼重和粉丝煲味道算OK，不过那个鱼腐太硬。必须吐槽一下服务！一个稍胖的女服务员爱理不理的，解释菜式时一副不屑的样子！', '0']
['Theplacewasgoodwasat29floorofplazaThefoodwetried:beeffriedrice..veryplaindoesn’thaveanyspecialtastenotevenanegginsidePorkbelly:nyummyworthtotryDurianpuffpastry:TOOsweet!Beefbraised:wasunderexpectationVegetable:so..sotheservicewasTERRIBLEThewaitressnotkind,doesn’thavewarmgreetingtoforeigner..Underexpectation..', '0']
['早上要11点才开门，排队又要在广场上排，大热天，等死人了，为什么不可以让人进入大堂去等候呢?须要改进', '0']
['应小伙伴的邀请，我评价一下＂空中一号

## 数据处理

为了将原始数据处理成模型可以读入的格式，本项目将对数据作以下处理：

- 首先使用jieba切词，之后将jieba切完后的单词映射词表中单词id。

![](https://ai-studio-static-online.cdn.bcebos.com/c538bbd04cb2489ab4ff260133247fa7ab8fb0da17874179bc320d773977cb5d)

- 使用`paddle.io.DataLoader`接口多线程异步加载数据。

其中用到了PaddleNLP中关于数据处理的API。PaddleNLP提供了许多关于NLP任务中构建有效的数据pipeline的常用API

| API                             | 简介                                       |
| ------------------------------- | :----------------------------------------- |
| `paddlenlp.data.Stack`          | 堆叠N个具有相同shape的输入数据来构建一个batch，它的输入必须具有相同的shape，输出便是这些输入的堆叠组成的batch数据。 |
| `paddlenlp.data.Pad`            | 堆叠N个输入数据来构建一个batch，每个输入数据将会被padding到N个输入数据中最大的长度 |
| `paddlenlp.data.Tuple`          | 将多个组batch的函数包装在一起 |

更多数据处理操作详见： [https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/data.md](https://github.com/PaddlePaddle/PaddleNLP/blob/develop/docs/data.md)

In [7]:
# 下载词汇表文件word_dict.txt，用于构造词-id映射关系。
!wget https://paddlenlp.bj.bcebos.com/data/senta_word_dict.txt

# 加载词表
vocab = load_vocab('./senta_word_dict.txt')

for k, v in vocab.items():
    print(k, v)
    break

--2021-03-22 19:44:22--  https://paddlenlp.bj.bcebos.com/data/senta_word_dict.txt
Resolving paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c00:6c21:10ad:0:ff:b00e:67d
Connecting to paddlenlp.bj.bcebos.com (paddlenlp.bj.bcebos.com)|182.61.200.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14600150 (14M) [text/plain]
Saving to: ‘senta_word_dict.txt’

senta_word_dict.txt 100%[===================>]  13.92M  29.7MB/s    in 0.5s    

2021-03-22 19:44:23 (29.7 MB/s) - ‘senta_word_dict.txt’ saved [14600150/14600150]

[PAD] 0


## 构造dataloder

下面的`create_data_loader`函数用于创建运行和预测时所需要的`DataLoader`对象。

* `paddle.io.DataLoader`返回一个迭代器，该迭代器根据`batch_sampler`指定的顺序迭代返回dataset数据。异步加载数据。

* `batch_sampler`：DataLoader通过 batch_sampler 产生的mini-batch索引列表来 dataset 中索引样本并组成mini-batch

* `collate_fn`：指定如何将样本列表组合为mini-batch数据。传给它参数需要是一个callable对象，需要实现对组建的batch的处理逻辑，并返回每个batch的数据。在这里传入的是`prepare_input`函数，对产生的数据进行pad操作，并返回实际长度等。

In [8]:
# Reads data and generates mini-batches.
def create_dataloader(dataset,
                      trans_function=None,
                      mode='train',
                      batch_size=1,
                      pad_token_id=0,
                      batchify_fn=None):
    if trans_function:
        dataset = dataset.apply(trans_function, lazy=True)

    # return_list 数据是否以list形式返回
    # collate_fn  指定如何将样本列表组合为mini-batch数据。传给它参数需要是一个callable对象，需要实现对组建的batch的处理逻辑，并返回每个batch的数据。在这里传入的是`prepare_input`函数，对产生的数据进行pad操作，并返回实际长度等。
    dataloader = paddle.io.DataLoader(
        dataset,
        return_list=True,
        batch_size=batch_size,
        collate_fn=batchify_fn)
        
    return dataloader

# python中的偏函数partial，把一个函数的某些参数固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。
trans_function = partial(
    convert_example,
    vocab=vocab,
    unk_token_id=vocab.get('[UNK]', 1),
    is_test=False)

# 将读入的数据batch化处理，便于模型batch化运算。
# batch中的每个句子将会padding到这个batch中的文本最大长度batch_max_seq_len。
# 当文本长度大于batch_max_seq时，将会截断到batch_max_seq_len；当文本长度小于batch_max_seq时，将会padding补齐到batch_max_seq_len.
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=vocab['[PAD]']),  # input_ids
    Stack(dtype="int64"),  # seq len
    Stack(dtype="int64")  # label
): [data for data in fn(samples)]


train_loader = create_dataloader(
    train_ds,
    trans_function=trans_function,
    batch_size=128,
    mode='train',
    batchify_fn=batchify_fn)
dev_loader = create_dataloader(
    dev_ds,
    trans_function=trans_function,
    batch_size=128,
    mode='validation',
    batchify_fn=batchify_fn)
test_loader = create_dataloader(
    test_ds,
    trans_function=trans_function,
    batch_size=128,
    mode='test',
    batchify_fn=batchify_fn)

# 模型搭建

使用`LSTMencoder`搭建一个BiLSTM模型用于进行句子建模，得到句子的向量表示。

然后接一个线性变换层，完成二分类任务。

- `paddle.nn.Embedding`组建word-embedding层
- `ppnlp.seq2vec.LSTMEncoder`组建句子建模层
- `paddle.nn.Linear`构造二分类器


<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/ecf309c20e5347399c55f1e067821daa088842fa46ad49be90de4933753cd3cf" width = "800" height = "450"  hspace='10'/> <br />
</p><br><center>图1：seq2vec示意图</center></br>

* 除LSTM外，`seq2vec`还提供了许多语义表征方法，详细可参考：[seq2vec介绍](https://aistudio.baidu.com/aistudio/projectdetail/1283423)

# LSTM

In [9]:
class LSTMModel(nn.Layer):
    def __init__(self,
                 vocab_size,
                 num_classes,
                 emb_dim=128,
                 padding_idx=0,
                 lstm_hidden_size=198,
                 direction='forward',
                 lstm_layers=1,
                 dropout_rate=0,
                 pooling_type=None,
                 fc_hidden_size=96):
        super().__init__()

        # 首先将输入word id 查表后映射成 word embedding
        self.embedder = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            padding_idx=padding_idx)

        # 将word embedding经过LSTMEncoder变换到文本语义表征空间中
        self.lstm_encoder = ppnlp.seq2vec.LSTMEncoder(
            emb_dim,
            lstm_hidden_size,
            num_layers=lstm_layers,
            direction=direction,
            dropout=dropout_rate,
            pooling_type=pooling_type)

        # LSTMEncoder.get_output_dim()方法可以获取经过encoder之后的文本表示hidden_size
        self.fc = nn.Linear(self.lstm_encoder.get_output_dim(), fc_hidden_size)

        # 最后的分类器
        self.output_layer = nn.Linear(fc_hidden_size, num_classes)

    def forward(self, text, seq_len):
        # text shape: (batch_size, num_tokens)
        # print('input :', text.shape)
        
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)
        # print('after word-embeding:', embedded_text.shape)

        # Shape: (batch_size, num_tokens, num_directions*lstm_hidden_size)
        # num_directions = 2 if direction is 'bidirectional' else 1
        text_repr = self.lstm_encoder(embedded_text, sequence_length=seq_len)
        # print('after lstm:', text_repr.shape)


        # Shape: (batch_size, fc_hidden_size)
        fc_out = paddle.tanh(self.fc(text_repr))
        # print('after Linear classifier:', fc_out.shape)

        # Shape: (batch_size, num_classes)
        logits = self.output_layer(fc_out)
        # print('output:', logits.shape)
        
        # probs 分类概率值
        probs = F.softmax(logits, axis=-1)
        # print('output probability:', probs.shape)
        return probs

model= LSTMModel(
        len(vocab),
        len(label_list),
        direction='bidirectional',
        padding_idx=vocab['[PAD]'])
model = paddle.Model(model)

# GRU模型

In [ ]:
class GRUModel(nn.Layer):
    def __init__(self,
                 vocab_size,
                 num_classes,
                 emb_dim=128,
                 padding_idx=0,
                 gru_hidden_size=198,
                 direction='forward',
                 gru_layers=1,
                 dropout_rate=0,
                 pooling_type=None,
                 fc_hidden_size=96):
        super().__init__()

        # 首先将输入word id 查表后映射成 word embedding
        self.embedder = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
            padding_idx=padding_idx)

        # 将word embedding经过GRUEncoder变换到文本语义表征空间中
        self.gru_encoder = ppnlp.seq2vec.GRUEncoder(
            emb_dim,
            gru_hidden_size,
            num_layers=gru_layers,
            direction=direction,
            dropout=dropout_rate,
            pooling_type=pooling_type)

        # GRUEncoder.get_output_dim()方法可以获取经过encoder之后的文本表示hidden_size
        self.fc = nn.Linear(self.gru_encoder.get_output_dim(), fc_hidden_size)

        # 最后的分类器
        self.output_layer = nn.Linear(fc_hidden_size, num_classes)

    def forward(self, text, seq_len):
        # text shape: (batch_size, num_tokens)
        # print('input :', text.shape)
        
        # Shape: (batch_size, num_tokens, embedding_dim)
        embedded_text = self.embedder(text)
        # print('after word-embeding:', embedded_text.shape)

        # Shape: (batch_size, num_tokens, num_directions*gru_hidden_size)
        # num_directions = 2 if direction is 'bidirectional' else 1
        text_repr = self.gru_encoder(embedded_text, sequence_length=seq_len)
        # print('after lstm:', text_repr.shape)


        # Shape: (batch_size, fc_hidden_size)
        fc_out = paddle.tanh(self.fc(text_repr))
        # print('after Linear classifier:', fc_out.shape)

        # Shape: (batch_size, num_classes)
        logits = self.output_layer(fc_out)
        # print('output:', logits.shape)
        
        # probs 分类概率值
        probs = F.softmax(logits, axis=-1)
        # print('output probability:', probs.shape)
        return probs

model= GRUModel(
        len(vocab),
        len(label_list),
        direction='bidirectional',
        padding_idx=vocab['[PAD]'])
model = paddle.Model(model)

# 模型配置和训练

## 模型配置

In [10]:
optimizer = paddle.optimizer.Adam(
        parameters=model.parameters(), learning_rate=5e-5)

loss = paddle.nn.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

model.prepare(optimizer, loss, metric)

In [11]:
# 设置visualdl路径
log_dir = './visualdl'
callback = paddle.callbacks.VisualDL(log_dir=log_dir)

## 模型训练

训练过程中会输出loss、acc等信息。这里设置了10个epoch，在训练集上准确率约97%。

![](https://ai-studio-static-online.cdn.bcebos.com/254cc9f80f474181a0f7fd00bb6f431502efdfdf54e54989a26549bc3abbe3c3)

In [12]:
model.fit(train_loader, 
            dev_loader,
            epochs=10, 
            save_dir='./checkpoints',
            save_freq=5, 
            callbacks=callback)

The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/10


Building prefix dict from the default dictionary ...
2021-03-22 19:45:52,963 - DEBUG - Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
2021-03-22 19:45:53,883 - DEBUG - Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.907 seconds.
2021-03-22 19:45:53,941 - DEBUG - Loading model cost 0.907 seconds.
Prefix dict has been built successfully.
2021-03-22 19:45:53,943 - DEBUG - Prefix dict has been built successfully.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step  10/251 - loss: 1.0981 - acc: 0.3477 - 239ms/step
step  20/251 - loss: 1.0957 - acc: 0.3477 - 167ms/step
step  30/251 - loss: 1.0974 - acc: 0.3453 - 139ms/step
step  40/251 - loss: 1.1007 - acc: 0.3385 - 127ms/step
step  50/251 - loss: 1.0972 - acc: 0.3377 - 126ms/step
step  60/251 - loss: 1.0997 - acc: 0.3367 - 122ms/step
step  70/251 - loss: 1.0964 - acc: 0.3363 - 119ms/step
step  80/251 - loss: 1.0973 - acc: 0.3394 - 117ms/step
step  90/251 - loss: 1.0952 - acc: 0.3407 - 114ms/step
step 100/251 - loss: 1.0980 - acc: 0.3416 - 113ms/step
step 110/251 - loss: 1.0964 - acc: 0.3413 - 112ms/step
step 120/251 - loss: 1.0966 - acc: 0.3432 - 111ms/step
step 130/251 - loss: 1.0931 - acc: 0.3424 - 112ms/step
step 140/251 - loss: 1.0932 - acc: 0.3429 - 111ms/step
step 150/251 - loss: 1.0974 - acc: 0.3427 - 110ms/step
step 160/251 - loss: 1.0886 - acc: 0.3438 - 109ms/step
step 170/251 - loss: 1.0929 - acc: 0.3433 - 109ms/step
step 180/251 - loss: 1.0814 - acc: 0.3450 - 109ms/step
step 190/2

## 启动VisualDL查看训练过程可视化结果
启动步骤：
- 1、切换到本界面左侧「可视化」
- 2、日志文件路径选择 'visualdl'
- 3、点击「启动VisualDL」后点击「打开VisualDL」，即可查看可视化结果：
Accuracy和Loss的实时变化趋势如下：
![](https://ai-studio-static-online.cdn.bcebos.com/cb5dff1e17e2407f91b83a1faabd09b4aaa7daac50f44d74a903a576452fbd09)

In [13]:
results = model.evaluate(dev_loader)
print("Finally test acc: %.5f" % results['acc'])

Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 10/31 - loss: 0.7828 - acc: 0.7445 - 93ms/step
step 20/31 - loss: 0.7629 - acc: 0.7457 - 79ms/step
step 30/31 - loss: 0.7550 - acc: 0.7536 - 68ms/step
step 31/31 - loss: 0.8319 - acc: 0.7523 - 66ms/step
Eval samples: 3968
Finally test acc: 0.75227


# 预测

In [14]:
label_map = {0: 'negative', 1: 'neutral',2:'positive'}
results = model.predict(test_loader, batch_size=128)[0]
predictions = []

for batch_probs in results:
    # 映射分类label
    idx = np.argmax(batch_probs, axis=-1)
    idx = idx.tolist()
    labels = [label_map[i] for i in idx]
    predictions.extend(labels)

# 看看预测数据前5个样例分类结果
for idx, data in enumerate(test_ds.data[:10]):
    print('Data: {} \t Label: {}'.format(data[0], predictions[idx]))

Predict begin...
step 32/32 [==============================] - ETA: 3s - 127ms/st - ETA: 3s - 119ms/st - ETA: 2s - 104ms/st - ETA: 2s - 107ms/st - ETA: 2s - 106ms/st - ETA: 2s - 102ms/st - ETA: 1s - 97ms/step - ETA: 1s - 94ms/ste - ETA: 1s - 91ms/ste - ETA: 1s - 88ms/ste - ETA: 0s - 86ms/ste - ETA: 0s - 84ms/ste - ETA: 0s - 83ms/ste - ETA: 0s - 79ms/ste - ETA: 0s - 76ms/ste - 72ms/step          
Predict samples: 3986
Data: 出品与环境都算可以吧，服务亦过得去。 	 Label: neutral
Data: 菜真是一般般滴，除了贵没啥优点… 	 Label: negative
Data: 第一次是朋友约的喝茶，两层楼都是坐满满客人，我们一行四个人，东西味道很正宗的广式茶点，最爱红米肠，还有乳鸽...网上团购真心觉得划算，在惠福路靠近北京路步行街总觉得东西会很贵，买单时让我感觉很意外四个人才一百多两百，旁边的一些客人也在说好划算哦，因为味道正宗，服务好，性价比高，这就是它为什么现在很多人选择它的原因，朋友家人聚餐的又一个不错的选择... 	 Label: neutral
Data: 抱住期待来…有d失望咯…点左bb猪，乳鸽松，上汤豆苗，薄撑，燕窝鹧鸪粥bb猪一般，无好好吃乳鸽松其实不错，不过实在太咸啦！！！！而且好多味精！吃完点饮水都唔够！上汤豆苗d豆苗唔知系乜豆苗，d汤底几好，不过都系咸左小小薄撑边系得得地啊，系唔多得！皮不够烟韧，又唔够脆，特別系在上边的，馅糖太多，花生系咸的！！！！！燕窝粥我无食…环境ok，就系有点嘈。服务一般，不太值10%。总结一句无乜动力令我再帮衬，实在太咸太甜啦！！！！！我宜家仲想买支水一口气喝掉半瓶！！！！！ 	 Label: neutral
Data: 点了一个小时的单，还没有上，上了的东西还是生的，跟服务员理论还一脸臭

这里只采用了一个基础的模型，就得到了较高的的准确率。

可以试试预训练模型，能得到更好的效果！参考[如何通过预训练模型Fine-tune下游任务](https://aistudio.baidu.com/aistudio/projectdetail/1294333)

# PaddleNLP 更多项目

 - [瞧瞧怎么使用PaddleNLP内置数据集-基于seq2vec的情感分析](https://aistudio.baidu.com/aistudio/projectdetail/1283423)
 - [如何通过预训练模型Fine-tune下游任务](https://aistudio.baidu.com/aistudio/projectdetail/1294333)
 - [使用BiGRU-CRF模型完成快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1317771)
 - [使用预训练模型ERNIE优化快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1329361)
 - [使用Seq2Seq模型完成自动对联](https://aistudio.baidu.com/aistudio/projectdetail/1321118)
 - [使用预训练模型ERNIE-GEN实现智能写诗](https://aistudio.baidu.com/aistudio/projectdetail/1339888)
 - [使用TCN网络完成新冠疫情病例数预测](https://aistudio.baidu.com/aistudio/projectdetail/1290873)
 - [使用预训练模型完成阅读理解](https://aistudio.baidu.com/aistudio/projectdetail/1339612)

# 加入交流群，一起学习吧

现在就加入PaddleNLP的QQ技术交流群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >